In [1]:
!pip install nba_api

     ---------------------------------------- 0.0/261.7 kB ? eta -:--:--
     - -------------------------------------- 10.2/261.7 kB ? eta -:--:--
     ---- -------------------------------- 30.7/261.7 kB 435.7 kB/s eta 0:00:01
     ---- -------------------------------- 30.7/261.7 kB 435.7 kB/s eta 0:00:01
     ---- -------------------------------- 30.7/261.7 kB 435.7 kB/s eta 0:00:01
     ----- ------------------------------- 41.0/261.7 kB 140.3 kB/s eta 0:00:02
     ---------- -------------------------- 71.7/261.7 kB 231.0 kB/s eta 0:00:01
     --------------- -------------------- 112.6/261.7 kB 312.2 kB/s eta 0:00:01
     --------------- -------------------- 112.6/261.7 kB 312.2 kB/s eta 0:00:01
     ------------------- ---------------- 143.4/261.7 kB 303.9 kB/s eta 0:00:01
     -------------------------- --------- 194.6/261.7 kB 380.0 kB/s eta 0:00:01
     ------------------------------ ----- 225.3/261.7 kB 404.2 kB/s eta 0:00:01
     -------------------------------- --- 235.5/261.7

In [1]:
import pandas as pd
import numpy as np
#import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

### Load Data
Fetch data from a free NBA Stat API

In [2]:
### Get All teams
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

In [3]:
from nba_api.stats.endpoints import leaguegamefinder

### Get games of current season
games_per_team = []
for team in nba_teams:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team['id'])
    games = gamefinder.get_data_frames()[0]
    games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
    games_per_team.append(games[(games.SEASON_ID == '22023') & (games.GAME_DATE >= '2023-10-24')])

games_per_team[0].head()


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612737,ATL,Atlanta Hawks,0022300393,2023-12-23,ATL vs. MEM,L,239,119,...,0.800,13,28,41,28,8.0,5,10,22,-6.0
1,22023,1610612737,ATL,Atlanta Hawks,0022300384,2023-12-22,ATL @ MIA,L,240,113,...,0.714,12,31,43,29,9.0,2,13,23,-9.0
2,22023,1610612737,ATL,Atlanta Hawks,0022300371,2023-12-20,ATL @ HOU,W,240,134,...,0.821,12,32,44,31,4.0,3,9,18,7.0
3,22023,1610612737,ATL,Atlanta Hawks,0022300352,2023-12-18,ATL vs. DET,W,240,130,...,0.815,11,29,40,25,6.0,10,17,24,6.0
4,22023,1610612737,ATL,Atlanta Hawks,0022300336,2023-12-16,ATL @ CLE,L,239,119,...,0.880,6,28,34,21,5.0,4,11,21,-8.0


In [61]:
from nba_api.stats.endpoints import boxscoreadvancedv3

### Get the mean of Offense, Defense and Net rating
target_stats = []
for team in games_per_team:
    WL = team.WL.value_counts().reset_index()
    W = WL[WL.WL == 'W']['count'].values[0]
    L = WL[WL.WL == 'L']['count'].values[0]
    target_stats.append({'team_abbreviation':team.TEAM_ABBREVIATION.unique()[0],
                         'team_name':team.TEAM_NAME.unique()[0],
                         'game_date':team.GAME_DATE.to_list(),
                         'W':W,
                         'L':L,
                         'offensive_rating':[], 
                         'defensive_rating':[],
                         'possessions':[] 
                         })
    
    for id in team.GAME_ID.to_list():
        stats = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=id) 
        if team[team.GAME_ID == id].MATCHUP.values[0][4] == "@":  ### Away Team
            target_stats[-1]['offensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['awayTeam']['statistics']['offensiveRating'])
            target_stats[-1]['defensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['awayTeam']['statistics']['defensiveRating'])
            target_stats[-1]['possessions'].append(stats.get_dict()['boxScoreAdvanced']['awayTeam']['statistics']['possessions'])
        else:
            target_stats[-1]['offensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['offensiveRating'])
            target_stats[-1]['defensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['defensiveRating'])
            target_stats[-1]['possessions'].append(stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['possessions'])
        
        

Important : Check if the team is hometeam or awayteam and get the right statistics

### Dashboard by Plotly

In [62]:
### Compute metrics
data = pd.DataFrame(target_stats)
data.offensive_rating = data.offensive_rating.apply(np.array)
data.defensive_rating = data.defensive_rating.apply(np.array)
data.possessions = data.possessions.apply(np.array)
data['GP'] = data.W + data.L
data['PCT_W'] = (data.W / data.GP)*100
data['PCT_W_rank'] = data.PCT_W.rank(method='min', ascending=False).astype('int64')
data.head()

,team_abbreviation,team_name,game_date,W,L,offensive_rating,defensive_rating,possessions,GP,PCT_W,PCT_W_rank
0,ATL,Atlanta Hawks,"[2023-12-23 00:00:00, 2023-12-22 00:00:00, 202...",12,17,"[119.0, 111.9, 135.4, 126.2, 122.7, 123.8, 125...","[125.0, 122.0, 128.3, 120.4, 130.9, 104.0, 133...","[100.0, 101.0, 99.0, 103.0, 97.0, 101.0, 102.0...",29,41.379310,22
1,BOS,Boston Celtics,"[2023-12-23 00:00:00, 2023-12-20 00:00:00, 202...",22,6,"[146.5, 151.6, 117.8, 115.2, 129.3, 119.6, 123...","[111.3, 124.0, 122.2, 99.0, 113.3, 111.5, 116....","[99.0, 95.0, 107.0, 99.0, 99.0, 97.0, 97.0, 99...",28,78.571429,1
2,CLE,Cleveland Cavaliers,"[2023-12-23 00:00:00, 2023-12-21 00:00:00, 202...",17,13,"[118.5, 113.0, 126.5, 131.1, 130.9, 111.5, 116...","[101.1, 130.9, 117.2, 126.2, 122.7, 119.6, 123...","[92.0, 92.0, 98.0, 103.0, 97.0, 96.0, 97.0, 92...",30,56.666667,13
3,NOP,New Orleans Pelicans,"[2023-12-23 00:00:00, 2023-12-21 00:00:00, 202...",17,13,"[110.6, 130.9, 117.7, 135.2, 112.0, 134.0, 119...","[110.4, 113.0, 118.6, 100.9, 108.1, 116.2, 104...","[94.0, 94.0, 96.0, 108.0, 100.0, 106.0, 101.0,...",30,56.666667,13
4,CHI,Chicago Bulls,"[2023-12-23 00:00:00, 2023-12-21 00:00:00, 202...",13,18,"[101.1, 116.3, 120.4, 113.7, 127.5, 126.5, 112...","[118.5, 96.0, 104.9, 108.3, 129.7, 117.2, 122....","[94.0, 98.0, 103.0, 95.0, 91.0, 98.0, 94.0, 10...",31,41.935484,21


In [63]:
### Data preparation for the dashboard
df = data
df.offensive_rating = df.offensive_rating.apply(np.mean).round(1)
df.defensive_rating = df.defensive_rating.apply(np.mean).round(1)
df['net_rating'] = df.offensive_rating - df.defensive_rating
df.net_rating = df.net_rating.round(1)
df['label'] = df.PCT_W_rank.astype('str') + '-' + df.team_abbreviation.astype('str')
df.sort_values(by='PCT_W_rank', inplace=True)

### Team colors
colors = pd.DataFrame({
    'team_abbreviation':['BOS', 'MIN', 'MIL', 'PHI', 'DEN', 'OKC', 'SAC', 'ORL', 'DAL',
       'LAC', 'MIA', 'NYK', 'CLE', 'NOP', 'HOU', 'LAL', 'GSW', 'IND',
       'PHX', 'BKN', 'CHI', 'ATL', 'UTA', 'TOR', 'MEM', 'CHA', 'POR',
       'WAS', 'SAS', 'DET'],
    'team_color':['#008248','#236192','#00471b','#006bb6','#0d2240','#007ac1','#5b2b82','#0b77bd','#007dc5',
                  '#1d428a','#98002e','#f58426','#6f2633','#b4975a','#ce1141','#552583','#fdb927','#002d62',
                  '#b95915','Black','#ce1141','#e03a3e','#2b5134','#a0a0a3','#5d76a9','#00788c','#cf0a2c',
                  '#002b5c','Black','#1d428a'
                  ]
})

df = df.merge(colors, on='team_abbreviation', how='left')

display(df.head())

,team_abbreviation,team_name,game_date,W,L,offensive_rating,defensive_rating,possessions,GP,PCT_W,PCT_W_rank,net_rating,label,team_color
0,BOS,Boston Celtics,"[2023-12-23 00:00:00, 2023-12-20 00:00:00, 202...",22,6,120.5,110.3,"[99.0, 95.0, 107.0, 99.0, 99.0, 97.0, 97.0, 99...",28,78.571429,1,10.2,1-BOS,#008248
1,MIN,Minnesota Timberwolves,"[2023-12-23 00:00:00, 2023-12-21 00:00:00, 202...",22,6,114.1,107.5,"[97.0, 102.0, 100.0, 97.0, 99.0, 101.0, 102.0,...",28,78.571429,1,6.6,1-MIN,#236192
2,MIL,Milwaukee Bucks,"[2023-12-23 00:00:00, 2023-12-21 00:00:00, 202...",22,7,120.5,115.0,"[103.0, 104.0, 110.0, 99.0, 104.0, 111.0, 109....",29,75.862069,3,5.5,3-MIL,#00471b
3,PHI,Philadelphia 76ers,"[2023-12-22 00:00:00, 2023-12-20 00:00:00, 202...",20,8,121.5,110.1,"[100.0, 101.0, 96.0, 98.0, 103.0, 103.0, 109.0...",28,71.428571,4,11.4,4-PHI,#006bb6
4,DEN,Denver Nuggets,"[2023-12-23 00:00:00, 2023-12-22 00:00:00, 202...",21,10,117.9,112.6,"[98.0, 95.0, 96.0, 96.0, 99.0, 103.0, 93.0, 99...",31,67.741935,5,5.3,5-DEN,#0d2240


In [66]:
### Plotly Configuration

### Scatter plot
scatter = go.Scatter(
    x=df['offensive_rating'],
    y=df['defensive_rating'],
    name='team',
    mode='markers+text',
    marker=dict(color=df['team_color'], size=7),
    #marker_symbol='diamond-wide',
    text=df['label'],
    textfont={'color':df['team_color'], 'size':7},
    textposition='top center',
    texttemplate='<b>[%{text}]</b>',
    hovertemplate = '%{text} <br>off_rtg: %{x} </br> def_rtg: %{y}'
)

### Table plot
table = go.Table(
    header=dict(values=list(['Team','rank','GP','W','L','off_r','def_r','net_r']),
                fill_color='lightblue',
                align='left'),
    cells=dict(values=[df['team_abbreviation'],
                       df['PCT_W_rank'],
                       df['GP'],
                       df['W'],
                       df['L'],
                       df['offensive_rating'],
                       df['defensive_rating'],
                       df['net_rating']
                       ],
               fill_color='snow',
               align='center'),
    domain=dict(x=[0.6, 1],
                y=[0, 1])
)

### Merge plots
layout = dict(xaxis1=dict( dict(domain=[0, 0.58], anchor='y1')),
              yaxis1=dict( dict(domain=[0, 1], anchor='x1')),
              margin=dict(l=50, r=30, t=45, b=50),
              title='NBA Teams Advanced Statistics'
             )
fig = go.Figure(data = [scatter,table], layout = layout)

### Configuration
fig.add_shape(type="line",
    x0=105, y0=105, x1=125, y1=125,
    line=dict(color="RoyalBlue",width=1)
)
fig['layout']['yaxis']['autorange'] = 'reversed'
fig.update_layout(xaxis_title='Offensive rating', 
                  yaxis_title='Defensive rating',
                  plot_bgcolor='whitesmoke',
                  font_family='Roboto',
                  font=dict(
                    size=11,
                    color='#044575'
                  )
                  )
iplot(fig, filename = 'basic_table')
